<a href="https://colab.research.google.com/github/ryannhongg/geneCorrelation.R/blob/main/%5Bshared%5D_OSD_513_performing_gsea_and_ora_capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
<img src="https://www.nasa.gov/wp-content/uploads/2024/07/osdr-gl4hs-logo.png" width="600"/>
</div>

# **NOTEBOOK: Performing gene set enrichment analysis (GSEA) and over-representation analysis (ORA)**
In this notebook, you will run statistical tests to determine which gene sets are over-represented by the differentially expressed genes you identified in the previous notebook. You'll also conduct gene set enrichment analysis which uses the gene expression data itself to determine which gene sets are enriched by genes differentially expressed between 2 groups (e.g. spaceflight and ground control).

## **Objectives of this notebook**
The primary objective of this notebook is to use Python's gseapy implementations of GSEA and ORA to find gene sets which are statistically significantly enriched by the genes that are differentially expressed between ground control and space flight groups in your dataset.

GSEA and ORA help us better understand which biological, cellular, and molecular processes and pathways explain the phenotypic differences we may observe between space flight and ground control mice. By extension, because the mouse is a model mammal organism which shares many body plans and orthologous genes with humans, we may infer how spaceflight may impact astronaut health. You can read more about GSEA in this [Wikipedia article](https://en.wikipedia.org/wiki/Gene_set_enrichment_analysis).

We use a library called enrichR which is written for R and Python, and has an [online portal](https://maayanlab.cloud/Enrichr/) you can query. You can learn more about enrichR and gene set enrichment analysis in this [YouTube video](https://www.youtube.com/watch?v=H1cUs6pql9s). The `gseapy` Python module has implementations for both GSEA as well as ORA. The major difference between these 2 approaches is that GSEA takes gene expression data as input and ORA takes a list of genes as input. As such, ORA is a more flexible tool and can be used to find gene sets and pathways  enriched by a list of genes regardless of how that list is derived. Here is a [Website](https://rnabio.org/module-08-scrna/0008/05/01/Gene_set_enrichment/) which discusses the differences between these 2 approaches.

## **UNIX commands introduced in this notebook**

N/A

# Prepare the runtime environment for the notebook

In [ ]:
# mount google drive
from google.colab import drive
drive.flush_and_unmount()
drive.mount("mnt")

Drive not mounted, so nothing to flush and unmount.
Mounted at mnt


In [ ]:
# time the notebook
import datetime
start_time = datetime.datetime.now()
print('notebook start time: ', start_time.strftime('%Y-%m-%d %H:%M:%S'))

notebook start time:  2025-08-19 23:58:51


In [ ]:
# define OSD dataset number to use for this notebook
OSD_DATASET='513'
GLDS_DATASET='513'

In [ ]:
# define factors for experiment

# young GC vs old GC
config_1 = {
    'experiment name': 'young_GC_vs_old_GC',
    'factor_list': ['Factor Value[Spaceflight]', 'Factor Value[Age]'],
    'group_A': {'Factor Value[Spaceflight]': 'Ground Control', 'Factor Value[Age]': '10 to 12'},
    'group_B': {'Factor Value[Spaceflight]': 'Ground Control', 'Factor Value[Age]': '32'}
}

# young FLT vs old FLT
config_2 = {
    'experiment name': 'young_FLT_vs_old_FLT',
    'factor_list': ['Factor Value[Spaceflight]', 'Factor Value[Age]'],
    'group_A': {'Factor Value[Spaceflight]': 'Space Flight', 'Factor Value[Age]': '10 to 12'},
    'group_B': {'Factor Value[Spaceflight]': 'Space Flight', 'Factor Value[Age]': '32'}
}

# young GC vs young FLT
config_3 = {
    'experiment name': 'young_GC_vs_young_FLT',
    'factor_list': ['Factor Value[Spaceflight]', 'Factor Value[Age]'],
    'group_A': {'Factor Value[Spaceflight]': 'Ground Control', 'Factor Value[Age]': '10 to 12'},
    'group_B': {'Factor Value[Spaceflight]': 'Space Flight', 'Factor Value[Age]': '10 to 12'}
}

# old GC vs old FLT
config_4 = {
    'experiment name': 'old_GC_vs_old_FLT',
    'factor_list': ['Factor Value[Spaceflight]', 'Factor Value[Age]'],
    'group_A': {'Factor Value[Spaceflight]': 'Ground Control', 'Factor Value[Age]': '32'},
    'group_B': {'Factor Value[Spaceflight]': 'Space Flight', 'Factor Value[Age]': '32'}
}

# FLT vs GC
config_5 = {
    'experiment name': 'FLT_vs_GC',
    'factor_list': ['Factor Value[Spaceflight]'],
    'group_A': {'Factor Value[Spaceflight]': 'Space Flight'},
    'group_B': {'Factor Value[Spaceflight]': 'Ground Control'}
}

# YNG vs OLD
config_6 = {
    'experiment name': 'YNG_vs_OLD',
    'factor_list': ['Factor Value[Age]'],
    'group_A': {'Factor Value[Age]': '10 to 12'},
    'group_B': {'Factor Value[Age]': '32'}
}

# LONG vs SHORT
config_7 = {
    'experiment name': 'LONG_vs_SHORT',
    'factor_list': ['Factor Value[Duration]'],
    'group_A': {'Factor Value[Duration]': '~25'},
    'group_B': {'Factor Value[Duration]': '~75'}
}
config = config_5


In [ ]:
# define output dir
OUTPUT_DIR="/content/drive/MyDrive/NASA/GL4HS/CAPSTONE/513/FLT_vs_GC"

In [ ]:
# set DGEA_DIR to get DEGs
import os
DGEA_DIR=OUTPUT_DIR + "/DGEA/"
if not os.path.exists(DGEA_DIR):
  raise Exception("STOP! You must finish the preceding notebook before running this one")

Exception: STOP! You must finish the preceding notebook before running this one

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
META_DIR=OUTPUT_DIR + "/META/"
if not os.path.exists(META_DIR):
  raise Exception("STOP! You must finish the preceding notebook before running this one")

In [ ]:
import os
GSEA_DIR=OUTPUT_DIR + "/GSEA/"
if os.path.exists(GSEA_DIR):
  !rm -rf {GSEA_DIR}
!mkdir -p {GSEA_DIR}

In [ ]:
# install packages and import the modules needed to do DGEA
!pip install gseapy==1.1.8 --no-cache
import gseapy as gp
from gseapy import Msigdb
!pip install mygene

# Define Python methods

In [ ]:
# define method to run ORA
def run_ora( gene_results, gene_set_list, background_genes=[], display_cols=['Term', 'Genes', 'Adjusted P-value', 'Combined Score'], sleep_time=5, retries=10):
  counter = 0
  import time
  enr = None
  gsea = None
  while counter < retries:
    try:
      # use enrichR to get gene ontology terms for molecular function, cellular components, and biological processes
      # sometimes the call fails, so just re-run the cell after a few seconds
      enr = gp.enrichr(gene_list=list(gene_results),
                      gene_sets=gene_set_list,
                      organism='mouse',
                      outdir=None,
                      )
    except:
      time.sleep(sleep_time)
      counter += 1
      continue
    break

  if not enr is None:
    # show top 10 results with adjusted p-value less than 0.05
    # other cols to display ['Term', 'Overlap', 'Genes', 'Adjusted P-value']
    gsea = enr.results[enr.results['Adjusted P-value'] < 0.05][display_cols]
  else:
    print('gsea failed')
  return gsea, enr

In [ ]:
# define method to run gseapy
def run_run_ora(deg_gene_symbols, background_gene_symbols, available_gene_sets, gene_set_list):
  gene_sets = list()
  for gene_set in gene_set_list:
    if gene_set not in available_gene_sets:
      print('gene set NOT found: ', gene_set)
    else:
      print('gene set found: ', gene_set)
      gene_sets.append(gene_set)
  results, enr = run_ora(deg_gene_symbols, gene_sets, background_gene_symbols)

  return results, enr

In [ ]:
# define method to convert gene ids to symbols
def get_symbols_from_ids(gene_list):
  import mygene

  symbol_list = list()
  mg = mygene.MyGeneInfo()
  ginfo = mg.querymany(gene_list, scopes='ensembl.gene')
  seen_genes = list()
  for g in ginfo:
    if g['query'] in seen_genes:
      continue
    if not 'symbol' in g:
      symbol_list.append(g['query'])
    else:
      symbol_list.append(g['symbol'])
    seen_genes.append(g['query'])
  return symbol_list

In [ ]:
# define method to transpose a dataframe
def transpose_df(df, cur_index_col, new_index_col):
  df = df.set_index(cur_index_col).T
  df.reset_index(level=0, inplace=True)
  cols = [new_index_col] + list(df.columns)[1:]
  df.columns = cols
  return df

# Read in data from google drive

In [ ]:
# read DEGs from file
import os
deg_gene_symbols = []
file_path = os.path.join(DGEA_DIR, 'deg_genes.txt')
if not os.path.exists(file_path):
  raise Exception("STOP! You must finish the preceding notebook before running this one")
with open(file_path, 'r') as f:
  for line in f:
    deg_gene_symbols.append(line.strip())

In [ ]:
# pring the first 10 degs
print('number of DEGs: ', len(deg_gene_symbols))
deg_gene_symbols[:10]

These genes are Ensembl gene symbols. You can learn more about Ensembl gene naming [here](https://www.informatics.jax.org/mgihome/nomen/gene.shtml).


We are going to submit the list of "background" genes along with the differentially expressed genes to the `gseapy` Enrichr tool. You can read more about background genes on this [biostars thread](https://www.biostars.org/p/17628/).

In [ ]:
# read background genes from file
background_gene_symbols = []
import os
file_path = os.path.join(DGEA_DIR, 'background_genes.txt')
if not os.path.exists(file_path):
  raise Exception("STOP! You must finish the preceding notebook before running this one")
with open(file_path, 'r') as f:
  for line in f:
    background_gene_symbols.append(line.strip())

In [ ]:
# print the first 10 background genes
print('number of background genes: ', len(background_gene_symbols))
background_gene_symbols[:10]

In [ ]:
# read in expr_df from google drive
import pandas as pd
expr_df = pd.read_csv(DGEA_DIR + '/expr_df_factors.csv')

# transpose dataframe so it's genes x samples
expr_df_t = transpose_df(expr_df, 'sample', 'Gene')

genes = list(expr_df_t['Gene'])

# convert gene ids to gene symbols
gene_symbols = get_symbols_from_ids(genes)

# then cast gene names to upper case
gene_symbols_upper = [gene.upper() for gene in gene_symbols]
expr_df_t['Gene'] = gene_symbols_upper

# print first 5 rows of expression matrix
expr_df_t.head()


# Run gene set enrichment analysis
We will use gsea from the `gseapy` Python package to perform gene set enrichment analysis. The code in this section was drived from [this example](https://gseapy.readthedocs.io/en/latest/gseapy_example.html#GSEA-Example) in the gseapy documentation.

In [ ]:
# read in factors from previous notebook
factors = pd.read_csv(DGEA_DIR + '/factors.csv')
classes = [str(i) for i in list(factors['factor'])]
print('factors shape: ', factors.shape)
print('classes: ', classes)


In [ ]:
# define gene sets to explore in GSEA
go_gene_sets = ['GO_Molecular_Function_2025', \
                 'GO_Biological_Process_2025', \
                 'GO_Cellular_Component_2025']

# run GSEA experiment
go_gsea_res = gp.gsea(data=expr_df_t,
                 gene_sets=go_gene_sets,
                 cls= classes,
                 # set permutation_type to phenotype if samples >=15
                 outdir=None,
                 method='signal_to_noise',
                 threads=2, seed= 7,
                 permutation_type='gene_set')

print(go_gsea_res)

# filter results based on FWER p-val < 0.05
go_gsea_results = go_gsea_res.res2d[go_gsea_res.res2d['FWER p-val'] < 0.05]

# define columns to display in output
display_cols = ['Term', 'Lead_genes', 'NES', 'FWER p-val']

# print head of GSEA results
print('gsea results: ', go_gsea_results[display_cols].head())

# save the GO GSEA results to google drive
go_gsea_results.to_csv(GSEA_DIR + '/go_gsea_results.csv', index=None)


In [ ]:
# plot GO GSEA results in dot plot
from gseapy import dotplot
import matplotlib.pyplot as plt

if go_gsea_results.empty:
  print('no GO GSEA results to plot')
else:
  ax = dotplot(go_gsea_results,
              column="FWER p-val",
              title='GO GSEA',
              cmap=plt.cm.viridis,
              size=5,
              figsize=(4,5), cutoff=1)

  # save dotplot to google drive
  ax.get_figure().savefig(GSEA_DIR + '/go_gsea_dotplot.png')

The smaller the FWER p-value, the more significant the statistical enrichment. The higher the percentage of genes in the gene set, the larger the dot in the plot. The horizontal axis is the normalized enrichment score (NES).

In [ ]:
# plot GO GSEA results in heatmap
from gseapy import heatmap
if go_gsea_results.empty:
  print('no GO GSEA results to plot')
else:
  terms = go_gsea_results.Term

  for i in range(len(go_gsea_results)):
    genes = go_gsea_results.Lead_genes[i].split(";")
    ax = heatmap(df = go_gsea_res.heatmat.loc[genes], z_score=0, title=terms[i], figsize=(14,4))

  # save heatmap data to google drive
  go_gsea_res.heatmat.to_csv(GSEA_DIR + '/go_gsea_heatmap.csv', index=None)

  # save heatmap figure to google drive
  ax.get_figure().savefig(GSEA_DIR + '/go_gsea_heatmap.png')

In [ ]:
geo_gene_sets = ['Disease_Perturbations_from_GEO_down', \
                 'Disease_Perturbations_from_GEO_up']

geo_gsea_res = gp.gsea(data=expr_df_t,
                 gene_sets=geo_gene_sets,
                 cls= classes,
                 # set permutation_type to phenotype if samples >=15
                 outdir=GSEA_DIR,
                 method='signal_to_noise',
                 threads=2, seed= 7,
                 permutation_type='gene_set')

# filter results based on FWER p-val
geo_gsea_results = geo_gsea_res.res2d[geo_gsea_res.res2d['FWER p-val'] < 0.05]

# define columns to display in output
display_cols = ['Term', 'Lead_genes', 'NES', 'FWER p-val']

# print top 10 GSEA results
print('head of geo gsea results: ', geo_gsea_results[display_cols].head())

# save the GO GSEA results to google drive
geo_gsea_results.to_csv(GSEA_DIR + '/geo_gsea_results.csv', index=None)

In [ ]:
# create the dot plot
from gseapy import dotplot
import matplotlib.pyplot as plt
if geo_gsea_results.empty:
  print('no GEO GSEA results to plot')
else:
  ax = dotplot(geo_gsea_results,
              column="FWER p-val",
              title='GEO GSEA',
              cmap=plt.cm.viridis,
              size=5,
              figsize=(4,5), cutoff=1)

  # save figure to google drive
  ax.get_figure().savefig(GSEA_DIR + '/geo_gsea_dotplot.png')

In [ ]:
# plot GEO GSEA results in heatmap
from gseapy import heatmap
if geo_gsea_results.empty:
  print('no GEO GSEA results to plot')
else:
  terms = geo_gsea_results.Term

  for i in range(len(geo_gsea_results)):
    genes = geo_gsea_results.Lead_genes[i].split(";")
    ax = heatmap(df = geo_gsea_res.heatmat.loc[genes], z_score=0, title=terms[i], figsize=(14,4))

  # save heatmap data to google drive
  geo_gsea_res.heatmat.to_csv(GSEA_DIR + '/geo_gsea_heatmap.csv', index=None)

  # save heatmap figure to google drive
  ax.get_figure().savefig(GSEA_DIR + '/geo_gsea_heatmap.png')

# Run over-representation analysis

We will use Enrichr from the `gseapy` Python package to perform over-representation analysis. You can learn more about Enrichr [here](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-14-128).

In [ ]:
# get list of all available gene sets to query
list_of_gene_sets=gp.get_library_name(organism='Mouse')
print('number of gene sets to analyze: ', len(list_of_gene_sets))
print(list_of_gene_sets)

We are only going to look at a few of those gene sets, but you should feel free to add more gene sets from the list above to the list below. You can learn more about these gene sets [here](https://maayanlab.cloud/Enrichr/#libraries).

In [ ]:
# verify the gene sets we're exploring are still part of the supported libraries
# if one is not available, terminate the notebook!
available_gene_sets = list()

for gene_set in ['GO_Molecular_Function_2025', \
                 'GO_Biological_Process_2025', \
                 'GO_Cellular_Component_2025', \
                 'Disease_Perturbations_from_GEO_down', \
                 'Disease_Perturbations_from_GEO_up']:
  if gene_set not in list_of_gene_sets:
    print('gene set not found!:', gene_set)
    raise Exception("STOP! remove this gene set from the gene_set list above as it appears to be unavailable")
  else:
    print('gene set found: ', gene_set)
    available_gene_sets.append(gene_set)

In [ ]:
# define dictionary to hold results and enrichR object to hold enrichR data
enrichr_results = {}
enrichr = {}

**NOTE**

The calls to the online Enrichr Web Service may fail (it's a free Web service that has been known to periodically fail). If they do fail, you can copy the list of differentially expressed genes into your favorite editor (like MS Word or textedit or Google Docs) and create a list such as the following by removing the `'`, `[`, `]` characters from the list. Then use online pathway and gene set enrichment tools such as [DAVID](https://davidbioinformatics.nih.gov/summary.jsp), [MSigDB](https://www.gsea-msigdb.org/gsea/login.jsp), and [ShinyGO](https://bioinformatics.sdstate.edu/go/).



In [ ]:
# Get GO results
# print the top 10
gene_sets = ['GO_Molecular_Function_2025','GO_Cellular_Component_2025', 'GO_Biological_Process_2025']
enrichr_results['GO'], enrichr['GO'] = run_run_ora(deg_gene_symbols, background_gene_symbols, available_gene_sets, gene_sets)
enrichr_results['GO'].head(10)

The output includes each of the gene ontologies that are over-represented by your list of significantly differentially expressed genes. The ORA output is sorted by adjusted p-value.

Each over-represented gene set includes your genes which belong to that gene set, along with the adjusted p-value representing the significance of the enrichment. Recall that, in DGEA where we are conducting the same statistical test thousands of times, we needed to adjust the p-value to account for the inflated number of false positives due to multiple testing. Similarly, here we are also conducting the same statistical test thousands of times -- once per gene set. So we also need to adjust the p-value accordingly. You can learn more about how p-values are calculated and adjusted in this [YouTube video](https://www.youtube.com/watch?v=EF94wPaqXM0).

You can learn more about GO (gene ontology) and the 3 aspects Molecular Function (MF), Cellular Component (CC), and Biological Process (BP) in [this document](https://geneontology.org/docs/ontology-documentation/).

In [ ]:
# get disease perturbations for down-regulated GEO signatures
# print the top 10
gene_sets = ['Disease_Perturbations_from_GEO_down']
enrichr_results['GEO_down'], enrichr['GEO_down'] = run_run_ora(deg_gene_symbols, background_gene_symbols, available_gene_sets, gene_sets)
enrichr_results['GEO_down'].head(10)


In [ ]:
# examine the full term and list of genes for the first enriched gene set in the GEO_down library.
if len(enrichr_results['GEO_down']) > 0:
  print('GEO_down term: ', enrichr_results['GEO_down'].iloc[0]['Term'])
  print('GEO_down genes: ', enrichr_results['GEO_down'].iloc[0]['Genes'])
else:
  print('no GEO_down results to print')

In [ ]:
# get disease perturbations for up-regulated GEO signatures
# print the top 10
gene_sets = ['Disease_Perturbations_from_GEO_up']
enrichr_results['GEO_up'], enrichr['GEO_up'] = run_run_ora(deg_gene_symbols, background_gene_symbols, available_gene_sets, gene_sets)
enrichr_results['GEO_up'].head(10)


In [ ]:
# examine the full term and list of genes for the first enriched gene set of the GEO_up library
if len(enrichr_results['GEO_up']) > 0:
  print('GEO_up term: ', enrichr_results['GEO_up'].iloc[0]['Term'])
  print('GEO_up genes: ', enrichr_results['GEO_up'].iloc[0]['Genes'])
else:
  print('no GEO_up results to print')

In [ ]:
# save results to file
for db in enrichr_results.keys():
  if not enrichr_results[db] is None:
    enrichr_results[db].to_csv(GSEA_DIR + '/ora_results_' + db + '.csv', index=None)

# Plot ORA results
A GO plot is used to visualize the enriched Gene Ontology (GO) terms generated from the previous steps of the notebook. The four plots generated below represent two of the same results in two slightly visual formats (dot plot versus lollipop plot): In both the dot plot and the lollipop plot, the size of the marker represents the significance of the over-representation (based on the adjusted p-value), while the position along the x-axis indicates the adjusted p-value itself, with smaller values (higher significance) appearing further to the right.  If a process is "significantly over-represented", this indicates that the set of genes is likely contributing to the differences between the 2 groups of your experiment.

In [ ]:
# Plot top GO terms as dot plot with size representing significance

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

if not enrichr_results['GO'].empty:
    # Add a column for plotting size based on -log10(Adjusted P-value)
    plot_data = enrichr['GO'].res2d.head(10).copy()
    plot_data['Significance'] = -np.log10(plot_data['Adjusted P-value'])

    plt.figure(figsize=(10, 8))
    sns.scatterplot(x='Adjusted P-value', y='Term', size='Significance', sizes=(100, 1000), \
                    data=plot_data, color='darkblue', legend=False) # sizes sets the range of dot sizes
    plt.title('Top 10 Enriched GO Terms (Size based on Significance)')
    plt.xlabel('Adjusted P-value')
    plt.ylabel('GO Term')
    plt.gca().invert_xaxis() # Invert x-axis to show smaller p-values at the top
    plt.savefig(GSEA_DIR + '/ora_go_dotplot.png')

    plt.show()
else:
    print("No significant GO terms found to plot.")

In [ ]:
# Plot top GEO up and down terms as dot plot with size representing significance
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

geo_results = pd.concat([enrichr_results['GEO_down'], enrichr_results['GEO_up']], ignore_index=True)

if not geo_results.empty:
    # Add a column for plotting size based on -log10(Adjusted P-value)
    plot_data = enrichr['GEO_down'].res2d.head(10).copy()
    plot_data['Significance'] = -np.log10(plot_data['Adjusted P-value'])

    plt.figure(figsize=(12, 10))
    sns.scatterplot(x='Adjusted P-value', y='Term', size='Significance', sizes=(100, 1000), \
                    data=plot_data, color='darkblue', legend=False) # sizes sets the range of dot sizes
    plt.title('Top 10 Enriched GEO Perturbation Signatures (Size based on Significance)')
    plt.xlabel('Adjusted P-value')
    plt.ylabel('GEO Term')
    plt.gca().invert_xaxis() # Invert x-axis to show smaller p-values at the top
    plt.savefig(GSEA_DIR + '/ora_geo_dotplot.png')

    plt.show()
else:
    print("No significant GEO terms found to plot.")

In [ ]:
# create lollipop plots for GO ORA results
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

if not enrichr_results['GO'].empty:
    '''plot_data = enrichr_results['GO'].head(10).copy()
    plot_data['Significance'] = -np.log10(plot_data['Adjusted P-value'])'''
    plot_data = enrichr['GO'].res2d.head(10).copy()
    plot_data['Significance'] = -np.log10(plot_data['Adjusted P-value'])
    plot_data = plot_data.sort_values('Adjusted P-value', ascending=True) # Sort for better visualization

    plt.figure(figsize=(10, 8))
    plt.hlines(y=plot_data['Term'], xmin=0, xmax=plot_data['Adjusted P-value'], color='skyblue')

    # Plot each point individually with its corresponding marker size
    for index, row in plot_data.iterrows():
        plt.plot(row['Adjusted P-value'], row['Term'], "o", markersize=row['Significance'] * 2, color='darkblue')

    plt.title('Top 10 Enriched GO Terms (Size based on Significance)')
    plt.xlabel('Adjusted P-value')
    plt.ylabel('GO Term')
    plt.gca().invert_xaxis() # Invert x-axis to show smaller p-values at the top
    plt.grid(axis='x', linestyle='--', alpha=0.7)
    plt.savefig(GSEA_DIR + '/ora_go_lollipop.png')

    plt.show()
else:
    print("No significant GO terms found to plot.")

In [ ]:
# create lollipop plots for GEO ORA results
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

geo_results = pd.concat([enrichr_results['GEO_down'], enrichr_results['GEO_up']], ignore_index=True)

if not geo_results.empty:
    plot_data = geo_results.head(10).copy()
    plot_data['Significance'] = -np.log10(plot_data['Adjusted P-value'])
    plot_data = plot_data.sort_values('Adjusted P-value', ascending=True) # Sort for better visualization

    plt.figure(figsize=(12, 10))
    plt.hlines(y=plot_data['Term'], xmin=0, xmax=plot_data['Adjusted P-value'], color='skyblue')

    # Plot each point individually with its corresponding marker size
    for index, row in plot_data.iterrows():
        plt.plot(row['Adjusted P-value'], row['Term'], "o", markersize=row['Significance'] * 2, color='darkblue')

    plt.title('Top 10 Enriched GEO Perturbation Signatures (Size based on Significance)')
    plt.xlabel('Adjusted P-value')
    plt.ylabel('GEO Term')
    plt.gca().invert_xaxis() # Invert x-axis to show smaller p-values at the top
    plt.grid(axis='x', linestyle='--', alpha=0.7)
    plt.savefig(GSEA_DIR + '/ora_geo_lollipop.png')
    plt.show()

else:
    print("No significant GEO terms found to plot.")

# Verify notebook is complete before moving on

In [ ]:
# make sure your google drive disk space utilization is still under 15G
!du -sh /content/mnt/MyDrive/NASA/GL4HS

In [ ]:
# time the notebook
import datetime
end_time = datetime.datetime.now()
print('notebook end time: ', end_time.strftime('%Y-%m-%d %H:%M:%S'))

print('notebook runtime: ', end_time - start_time)